In [33]:
import pandas as pd
import os
import re


carpeta = "Data encontrada"

# Lista para almacenar cada DataFrame
dataframes = []

#dff = pd.read_excel('Data encontrada/resultados_inalco.xlsx')

for archivo in os.listdir(carpeta):
    if archivo.endswith(".xlsx") or archivo.endswith(".xls"):
        ruta = os.path.join(carpeta, archivo)
        #print(f"Leyendo: {archivo}")
        df = pd.read_excel(ruta)
        nombre_archivo = os.path.splitext(archivo)[0]  # Quita la extensión
        df["Pagina"] = nombre_archivo.replace("resultados_", "")
        df["Pagina"] = df["Pagina"].str.upper()
        dataframes.append(df)

df_concatenado = pd.concat(dataframes, ignore_index=True)


In [ ]:
# df_takora = pd.read_excel('Data encontrada/resultados_takora.xlsx')

# df_takora['Nombre Normalizado'] = (
#     df_takora['Nombre Producto']
#       .astype(str)
#       # split por •, , , ( o la palabra Medida
#       .str.split(r'(?:•|,|\(|/|Medida)', n=1, regex=True)
#       .str[0]
#       .str.strip()
# )

LIMPIEZA DE DATOS

In [34]:
import unicodedata

def quitar_acentos(texto: str) -> str:
    nkfd = unicodedata.normalize('NFD', str(texto))
    return ''.join([c for c in nkfd if not unicodedata.combining(c)])

df_concatenado = df_concatenado.drop_duplicates(subset=[ 'Link']) #'Nombre Producto',


df_concatenado['Precio'] = (
    df_concatenado['Precio']
    .astype(str)
    .str.replace('$', '', regex=False)
    .str.replace('.', '', regex=False)
    .str.replace(',', '', regex=False)
    .str.strip()
)

df_concatenado['Precio'] = pd.to_numeric(df_concatenado['Precio'], errors='coerce')

df_precios_cero = df_concatenado[df_concatenado['Precio'].isna() | (df_concatenado['Precio'] == 0)].copy()
df_concatenado = df_concatenado[~df_concatenado.index.isin(df_precios_cero.index)].copy()

#QUITAR DATA DE CIPER
df_ciper = df_concatenado[df_concatenado['Pagina'] == 'CIPER'].copy()
df_concatenado = df_concatenado[~df_concatenado['Pagina'].isin(['CIPER'])].copy()

mask_dup = df_concatenado.duplicated(subset=['Nombre Producto'], keep=False)
df_nombres_duplicados = df_concatenado[mask_dup].copy()
df_concatenado = df_concatenado[~mask_dup].copy()
df_concatenado['Nombre Producto'] = df_concatenado['Nombre Producto'].apply(quitar_acentos)


conteos = df_nombres_duplicados['Pagina'].value_counts()
print(conteos)



# df_nombres_repetidos = df_nombres_duplicados[~df_nombres_duplicados["Pagina"].isin(["MUNDOREPUESTOS","CHILEREPUESTOS","CIPER",])]
# df_nombres_duplicados = df_nombres_duplicados[df_nombres_duplicados["Pagina"].isin(["MUNDOREPUESTOS","CHILEREPUESTOS","CIPER",])] 

Pagina
CHILEREPUESTOS     4737
MUNDOREPUESTOS     4251
TAKORA              636
INALCO              296
CASADEREPUESTOS     194
AUTOPLANET          130
EMGI                 40
ULTI                 18
Name: count, dtype: int64


In [47]:
conteos = df_concatenado['Pagina'].value_counts()
print(conteos)

Pagina
TAKORA            5391
AUTOPLANET        2604
INALCO             863
EMGI               850
CHILEREPUESTOS     705
MUNDOREPUESTOS     638
SALFAREPUESTOS     386
CIPER              297
ULTI                18
Name: count, dtype: int64


In [35]:
marcas= [
    'Chevrolet', 'Hyundai', 'Nissan', 'Toyota', 'Kia',
    'Susuki', 'Peugeot', 'Ford', 'Mazda', 'Mitsubishi'
]

modelos= [
    'Spark', 'Aveo', 'Sail', 'Cruze', 'Malibu', 'Tracker', 'Captiva', 'S10',
    'Accent', 'Elantra', 'Sonata', 'Tucson', 'Santa Fe', 'Creta', 'i10', 'Kona',
    'Sentra', 'Altima', 'Versa', 'March', 'X-Trail', 'Pathfinder', 'Frontier', 'Z Series',
    'Corolla', 'Camry', 'Yaris', 'Hilux', 'RAV4', 'Land Cruiser', 'Prius', 'Supra',
    'Picanto', 'Rio', 'Cerato', 'Sportage', 'Sorento', 'Soul', 'Seltos', 'Carnival',
    'Swift', 'Alto', 'Baleno', 'Celerio', 'Vitara', 'Jimny', 'SX4', 'Ertiga',
    '208', '308', '508', '2008', '3008', '5008', 'Partner', 'Rifter',
    'Fiesta', 'Focus', 'Mustang', 'Ranger', 'F-150', 'Escape', 'Explorer', 'Maverick',
    'Mazda2', 'Mazda3', 'Mazda6', 'CX-3', 'CX-5', 'CX-9', 'BT-50', 'MX-5',
    'Lancer', 'Mirage', 'Outlander', 'Outlander Sport', 'ASX', 'Eclipse Cross', 'Montero Sport', 'L200'
]


df_dicc_ciper = pd.read_excel('Data consolidada/marca_modelo_ciper2.xlsx')

df_ciper = df_ciper.drop(['Marca Buscada', 'Modelo Buscado'], axis=1)


df_ciper = pd.merge(
    df_ciper,
    df_dicc_ciper,
    on='Link',
    how='left',
    validate='many_to_one'
)


df_ciper.rename(
    columns={
        'Marca': 'Marca Buscada',
        'Modelo': 'Modelo Buscado'
    },
    inplace=True  # modifica df directamente
)

df_ulti = df_concatenado[df_concatenado['Pagina'] == 'ULTI'].copy()
df_inalco = df_concatenado[df_concatenado['Pagina'] == 'INALCO'].copy()
df_valido = df_concatenado[~df_concatenado['Pagina'].isin(['ULTI', 'INALCO'])].copy()

df_ulti_ciper = pd.concat([df_ulti, df_ciper], ignore_index=True)

def extraer_por_contains(texto, keywords):
    texto = str(texto).lower()
    for kw in keywords:
        if kw.lower() in texto:
            return kw
    return None

# Aplica sobre tu df_ciper
df_ulti_ciper['Marca Buscada'] = df_ulti_ciper['Marca Buscada'] \
    .apply(lambda x: extraer_por_contains(x, marcas))

df_ulti_ciper['Modelo Buscado'] = df_ulti_ciper['Modelo Buscado'] \
    .apply(lambda x: extraer_por_contains(x, modelos))

df_ulti_ciper.dropna(
    subset=['Marca Buscada', 'Modelo Buscado'],
    how='any',
    inplace=True
)

GUARDANDO ARCHIVOS

In [ ]:

def normalize(text):
    clean = re.sub(r'[^0-9a-z]+', ' ', text.lower())
    return re.sub(r'\s+', ' ', clean).strip()



marcas_set  = set(marcas)
modelos_set = set(modelos)



#BUSCA LA MARCA Y EL MODELO EN EL NOMBRE DEL REPUESTO
def contiene_marca_modelo_por_listas(row):
    nombre = normalize(row["Nombre Producto"])
    encontrada_marca = any(normalize(m) in nombre for m in marcas_set)
    encontrado_modelo = any(normalize(m) in nombre for m in modelos_set)
    return encontrada_marca and encontrado_modelo



df_valido = df_concatenado[df_concatenado.apply(contiene_marca_modelo_por_listas, axis=1)].copy()
df_no_valido = df_concatenado[~df_concatenado.apply(contiene_marca_modelo_por_listas, axis=1)].copy()

df_valido = pd.concat([df_valido, df_inalco, df_ulti_ciper], ignore_index=True)

df_repuestos = pd.read_csv('Tipos repuestos/tipos_repuestos.csv')
lista_repuestos = df_repuestos['repuestos'].dropna().tolist()

lista_repuestos = [str(r).strip().lower() for r in lista_repuestos]

def detectar_tipo_repuesto(nombre_producto):
    nombre = str(nombre_producto).lower()
    for repuesto in lista_repuestos:
        if repuesto in nombre:
            return repuesto
    return None



# Aplicar la función
df_valido['tipo_repuesto'] = df_valido['Nombre Producto'].apply(detectar_tipo_repuesto)


df_con_tipo = df_valido[df_valido['tipo_repuesto'].notna()].copy()


mask_takora = df_con_tipo['Pagina'].str.contains('TAKORA', case=False, na=False)
df_con_tipo.loc[mask_takora, 'Nombre Producto'] = (
    df_con_tipo.loc[mask_takora, 'Nombre Producto']
      .astype(str)
      .str.split(r'(?:•|,|\(|/|Medida)', n=1, regex=True)
      .str[0]
      .str.strip()
)


columnas_necesarias = ['Nombre Producto', 'Marca Buscada', 'Modelo Buscado','tipo_repuesto', 'Precio', 'Pagina', 'Link', 'Imagen', 'Descripción', 'fecha_carga']
df_con_tipo = df_con_tipo[columnas_necesarias].copy()

# Paso 2: Renombrar columnas
nuevos_nombres = {
    'Nombre Producto': 'nombre',
    'Marca Buscada': 'marca',
    'Modelo Buscado': 'modelo',
    'Precio': 'precio',
    'Pagina': 'pagina',
    'Link': 'link',
    'Imagen': 'imagen',
    'Descripción': 'descripcion',
    'fecha_carga' : 'fecha_carga'
}
df_con_tipo.rename(columns=nuevos_nombres, inplace=True)
df_con_tipo.to_excel('data_carga.xlsx', index=False)



In [37]:
conteos = df_con_tipo['pagina'].value_counts()
print(conteos)

pagina
TAKORA            3324
CIPER              974
INALCO             800
AUTOPLANET         481
MUNDOREPUESTOS     449
EMGI               402
CHILEREPUESTOS     271
ULTI                18
Name: count, dtype: int64
